# Обучаем нейронную сеть на тензорном процессоре Google TPU


In [ ]:
import numpy as np
import tensorflow as tf
import os
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.layers import Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing import image
from scipy.misc import toimage
from PIL import Image

import matplotlib.pyplot as plt
%matplotlib inline 


In [ ]:
# Названия классов из набора данных CIFAR-10
classes=['самолет', 'автомобиль', 'птица', 'кот', 'олень', 'собака', 'лягушка', 'лошадь', 'корабль', 'грузовик']

**Загружаем данные**

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

**Просмотр примеров данных**

In [ ]:
n = 1703
plt.imshow(Image.fromarray(x_train[n]))
plt.show()
print("Номер класса:", y_train[n])
print("Тип объекта:", classes[y_train[n][0]])

## Создаем нейронную сеть

In [ ]:
def create_model():
    input_layer = Input(shape=(32, 32, 3), dtype=tf.float32, name='Input')
    x = BatchNormalization()(input_layer)
    x = Conv2D(32, (3, 3), padding='same', activation='relu')(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.25)(x)
    x = BatchNormalization()(x)
    x = Conv2D(64, (3, 3), padding='same', activation='relu')(x)
    x = Conv2D(64, (3, 3), activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.25)(x)
    x = Flatten()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    output_layer = Dense(10, activation='softmax')(x)
    model = Model(inputs=[input_layer], outputs=[output_layer])
    model.compile(
        optimizer=tf.train.AdamOptimizer(0.001),
        loss=tf.keras.losses.sparse_categorical_crossentropy,
        metrics=['sparse_categorical_accuracy'])
    return model   

In [ ]:
cpu_model = create_model()

## Применяем сеть для распознавания объектов на CPU

Создаем модель на CPU и загружаем веса в нее

In [ ]:
model = create_model()

In [ ]:
model.load_weights("cifar10_model.h5")

In [ ]:
model.summary()

**Просмотр изображения из набора данных для тестирования**

In [ ]:
index=11

plt.imshow(Image.fromarray(x_test[index]))
plt.show()

#plt.imshow(toimage(x_test[index]))
#plt.show()

**Преобразование тестового изображения**

In [ ]:
x = x_test[index]
x = np.expand_dims(x, axis=0)

**Запуск распознавания**

In [ ]:
prediction = model.predict(x)

**Печатаем результаты распознавания**

In [ ]:
print(prediction)
prediction = np.argmax(prediction)
#print(classes[prediction])

In [ ]:
print(classes[prediction])